In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "baseline"
MODEL_NAME = "baseline"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000

# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [9]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

Updated property [core/project].


In [10]:
train_df = pd.read_csv("gs://fb-congressional-data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
dev_df = pd.read_csv("gs://fb-congressional-data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [0]:
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [17]:
time_start = time.time()

tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 11 minutes, 03 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [20]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]   98.9 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
\ [5 files][  2.9 GiB/  2.9 GiB]   27.3 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

0.15381079935942088

In [0]:
# embedding_matrix.shape

(1113992, 100)

In [22]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           111399200 
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
# Set up oversampling with SMOTE (Synthetic Minority Oversampling Technique)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1.0) # Willl bring final ratio to 1.0
x_train_res, y_train_res = sm.fit_sample(smaller_X_train, smaller_y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [0]:
try:
  time_start = time.time()

  history = model.fit(smaller_X_train, smaller_y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 4000000 samples, validate on 1000000 samples
Epoch 1/30
4000000/4000000 [==============================] - 103s 26us/sample - loss: 0.4809 - acc: 0.7717 - val_loss: 0.4742 - val_acc: 0.8173
Epoch 2/30
4000000/4000000 [==============================] - 101s 25us/sample - loss: 0.4804 - acc: 0.7718 - val_loss: 0.4662 - val_acc: 0.8198
Epoch 3/30
4000000/4000000 [==============================] - 102s 25us/sample - loss: 0.4800 - acc: 0.7720 - val_loss: 0.4722 - val_acc: 0.8192
Epoch 4/30
4000000/4000000 [==============================] - 102s 26us/sample - loss: 0.4795 - acc: 0.7723 - val_loss: 0.4767 - val_acc: 0.8142
Epoch 5/30
4000000/4000000 [==============================] - 103s 26us/sample - loss: 0.4792 - acc: 0.7725 - val_loss: 0.4683 - val_acc: 0.8197
Epoch 6/30
4000000/4000000 [==============================] - 101s 25us/sample - loss: 0.4788 - acc: 0.7726 - val_loss: 0.4705 - val_acc: 0.8190
Epoch 7/30
4000000/4000000 [==============================] - 100s 25us/samp

In [0]:
loss, accuracy = model.evaluate(smaller_X_train, smaller_y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7765
Testing Accuracy:  0.8178


In [0]:
# can't get this to work
plot_model(history)

In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [0]:
wrong_preds.op_gender.value_counts()

W    296541
M    115388
Name: op_gender, dtype: int64

In [0]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.07341074016521386

In [0]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,preds,probs
1977975,11027367,86569077,W,389827,Dan,"I asked Senator Mark Kirk to support this and he did not. Very disappointed, considering he is m...",Kirsten Gillibrand,Congress_Democratic,M,0.861726
2005426,11054818,86569077,W,390085,Keith,POS! I knew she was a democrap turncoat,Kirsten Gillibrand,Congress_Democratic,M,0.692199
2087640,11228620,77234050,W,399295,Audrey,"yes you did get stories and they were very postive,we as Americans want and deserve ACA. you hav...",Cathy McMorris Rodgers,Congress_Republican,M,0.805755
239790,2442996,71417411,M,54820,Jay,Why do the people of Planned Parenthood lie so much ?,Blake Farenthold,Congress_Republican,W,0.479726
1936248,10985640,86569077,W,389409,Rob,Will you also please mention her illegal and scandalous history ?,Kirsten Gillibrand,Congress_Democratic,M,0.617255
1981930,11031322,86569077,W,389864,Rowland,You're a powerhouse. Love it!,Kirsten Gillibrand,Congress_Democratic,M,0.554559
397524,3147615,23984618,W,83716,Mike,More nonsense!!!,Ann Wagner,Congress_Republican,M,0.683614
1244834,7371402,94431309,W,277104,Beth,"I am truly happy for your family. However, your hypocrisy is mind-boggling. You, of all people...",Jaime Herrera Beutler,Congress_Republican,M,0.619583
2077148,11218128,77234050,W,398827,Blainer,Are you drinking from the snake river?,Cathy McMorris Rodgers,Congress_Republican,M,0.800732
2000187,11049579,86569077,W,390039,Mike,Linda. Have a common name that gets you delayed by NICS? That's an infringement. As MLK said ...,Kirsten Gillibrand,Congress_Democratic,M,0.771842


In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,preds,probs
2137926,11278906,77234050,W,399910,Doug,I refuse to watch that lying POS on TV anymore. I refuse to watch the clapping seals seated in ...,Cathy McMorris Rodgers,Congress_Republican,M,0.999883
1993458,11042850,86569077,W,389980,Kristin,"That is why I stand by Rand. But you must be constitutional across the bridge and back, not sel...",Kirsten Gillibrand,Congress_Democratic,M,0.999851
1957878,11007270,86569077,W,389621,Amy,I STAND WITH RAND!! 2016,Kirsten Gillibrand,Congress_Democratic,M,0.999807
2151486,11292466,77234050,W,400340,Rod,"Thanks, Rep. McMorris for your continuous, unflinching fight against this oppressively Socialist...",Cathy McMorris Rodgers,Congress_Republican,M,0.999803
2001592,11050984,86569077,W,390046,Reaver,Stand with Rand /Cruz/Lee,Kirsten Gillibrand,Congress_Democratic,M,0.999792
2003832,11053224,86569077,W,390071,Susan,"Yes, thank goodness for Rand Paul! Wasting the tax payers money!",Kirsten Gillibrand,Congress_Democratic,M,0.999785
2037175,11086567,86569077,W,390472,Matt,http://www.slate.com/blogs/blogs/weigel/archive/2011/05/26/rand-paul-s-noble-defeat-on-the-patri...,Kirsten Gillibrand,Congress_Democratic,M,0.999782
1993131,11042523,86569077,W,389977,Colleen,You mean Mr Paul Mark? Rand Opportunist Paul?,Kirsten Gillibrand,Congress_Democratic,M,0.999778
429400,3362635,76269753,W,103747,Ivan,Do you stand with Rand?,Dianne Feinstein,Congress_Democratic,M,0.999776
429376,3362611,76269753,W,103747,Shane,Do you Stand with Rand?,Dianne Feinstein,Congress_Democratic,M,0.999776


In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,preds,probs
1887369,10920297,10458808,M,386070,Stephen,Yes yes yes. Kamala. You have always been there for us!!! Fantastic!,Mark Takano,Congress_Democratic,W,0.000009
1667284,10169799,76267383,M,374064,Mary,God bless you Elizabeth! You are an inspiration!,Keith Rothfus,Congress_Republican,W,0.000090
1604123,10051993,49028900,M,365938,Dana,"Well, as opposed to worrying about calories, let's do focus on the nutritious part. Let's face i...",Steve King,Congress_Republican,W,0.000091
181521,2358273,62246450,M,52502,Annie,Keep going Elizabeth! We support you!,Ron Wyden,Congress_Democratic,W,0.000152
264413,2528948,59227815,M,63410,Rhonda,"Love you, Mia Love! Almost-aborted first Black congresswoman.",Paul Ryan,Congress_Republican,W,0.000266
196738,2373490,62246450,M,52658,Randy,"Yay! 8-) Call Senator Warren, too!",Ron Wyden,Congress_Democratic,W,0.000315
900689,5149743,46506154,M,161080,Pam,Someone take away Patty Murray's tennis shoes. She is a thorn. Everyone knows that the Supre...,Marco Rubio,Congress_Republican,W,0.000364
1703132,10208736,32185537,M,376493,Kim,Patty Murray and the Green machine are working on doing this here in WA as well.,Paul Gosar,Congress_Republican,W,0.000378
1065864,5314918,46506154,M,161457,Susan,Love Mia - pulling for her to have a huge win in Utah - we need you in Congress girl.,Marco Rubio,Congress_Republican,W,0.000431
901117,5150171,46506154,M,161080,Sandy,There she is.....Patty Murray........dumber than a box of rocks!,Marco Rubio,Congress_Republican,W,0.000432
